In [0]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

In [0]:
def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  regex = re.compile('[^A-Za-zА-Яа-я]')
  cleantext = regex.sub(' ', cleantext)
  return cleantext

In [0]:
def gettext(bad): 
    mid = []
    i = 1
    while bad[i].text.split()[0] != 'Материал' and bad[i].text.split()[1] != 'опубликован':
        mid.append(bad[i].text)
        i += 1
        if i > len(mid):
            break
    good = [j for i in mid for j in i.split()]
    return good

In [0]:
def getli(li):
    mid = []
    

In [0]:
##вроде работает для этого класса, но делает лист с листами 
page = requests.get('https://vc.ru/15000')
soup = BeautifulSoup(page.text, 'html5lib')
info = []
text = []
for item in soup.find_all('div',  class_="layout--с block-html"):
    name = item.find_all('p')
    for i  in name:
      text.append(i.text.split())

In [0]:
##теперь и для этого класса
for item in soup.find_all('div',  class_="layout--a"):
    name = item.find_all('p')
    for i  in name:
      if i.find('span', class_="user_name"):
        break
      text.append(i.text)
text = text[:-2]
info.append(text)

In [0]:
def document(id):
    page = requests.get('https://vc.ru/' + str(id) )
    soup = BeautifulSoup(page.text, 'html5lib')
    info = []
    text =[]
    lol = []
    if soup.find("h1", class_="content-header__title") and 'Вакансия' not in soup.find('title').text:
        #titledone
        info.append(soup.find("h1", class_="content-header__title").text.split())
        #text
        if soup.find_all('p') != None:
          for item in soup.find_all('div',  class_="layout--с block-html"):
            name = item.find_all('p')
            for i  in name:
              text.append(i.text)
          for item in soup.find_all('div',  class_="layout--a"):
              name = item.find_all('p')
              for i  in name:
                if i.find('span', class_="user_name"):
                  break
                lol.append(i.text)
          lol = lol[:-2]
          text.append(lol)
        if soup.find_all("li") != None: 
            text.append([var for i in soup.find_all("li") for var in i.text.split()])
        else:
            text.append('NaN')
        info.append(text)
        #yeardone
        info.append(soup.find("time", class_= "time").text.split()[2])
        #sectiondone
        info.append(soup.find("div", class_=
                              "content-header-author__name").text.split())
        #votesdone
        a = soup.find("span", class_="vote__value__v vote__value__v--real").text
        if a[0] == '–':
            b = int(a[1:])
            info.append(-b)
        else:
            info.append(int(a))
        #viewsdone
        info.append(soup.find("span", class_="views__value").text.replace(u'\xa0', u''))
        #bookmarksdone
        if soup.find('div',
                        class_="favorite_marker favorite_marker--type-content favorite_marker--non_zero"):
            info.append(int(soup.find('div',
                        class_="favorite_marker favorite_marker--type-content favorite_marker--non_zero").text.split()[0]))
        else:
            info.append(0)
        #commentsdone
        comment1 = soup.find_all("div", class_="comments__item__text")
        comments = []
        for i in range(len(comment1)):
        if comment1[i].text != 'Комментарий удален':
            comments.append(comment1[i].text.split())
            info.append(comments)
    else:
        info.append('NaN')
    print(id)
    return info

In [0]:
from multiprocessing import Pool
with Pool(100) as p:
    a = p.map(document, np.arange(3246,3346))

##### Материал опубликован редакцией, а не автором

In [0]:
document(3256)

In [0]:
df = pd.DataFrame(a[1], columns=['title', 'text', 'date', 'section', 'votes', 'views', 'bookmarks', 'comments'])

In [0]:
a[:8]

In [0]:
i = 239400
for j in range(239500, 351001, 100):
    from multiprocessing import Pool
    with Pool(100) as p:
        b = p.map(document, np.arange(i, j))
    try:
        df = pd.DataFrame(b, columns=['title', 'text', 'date', 'hubs', 'tags', 'votes', 'views', 'bookmarks', 'comments'])
        df.to_feather(str(i) + "-" + str(j) + '.feather')
        from google.colab import files
        files.download(str(i) + "-" + str(j) + '.feather')
        print('done', i, j)
    except:
        print('ooops')
    i += 100

In [0]:
df = pd.DataFrame(a, columns=['title', 'text', 'year', 'section', 'votes', 'views', 'bookmarks', 'comments'])
df.dropna(inplace=True)
df

TypeError: ignored

In [0]:
df.to_csv('vcru.csv')
from google.colab import drive
drive.mount('/content/drive')

from google.colab import files
files.download('vcru.csv')

In [0]:
a = document(6832)
len(a[-1])

6832


71